# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']


F. Walter  ->  F. Walter  |  ['F. Walter']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']
Arxiv has 57 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2308.16239


extracting tarball to tmp_2308.16239...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:473: LatexWarning: Error parsing the document directly. Trying to recover.
  warnings.warn(LatexWarning(f"Error parsing the document directly. Trying to recover."))


...

extracting tarball to tmp_2308.16895...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2308.16895/boogaard-miri-hudf-alma.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'sources-final.tex' from 'tmp_2308.16895/sources-final.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


F. Walter  ->  F. Walter  |  ['F. Walter']
S. Bosman  ->  S. Bosman  |  ['S. Bosman']


Found 80 bibliographic references in tmp_2308.16895/boogaard-miri-hudf-alma.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.16895-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.16895) | **MIDIS: JWST/MIRI reveals the Stellar Structure of ALMA-selected Galaxies  in the Hubble-UDF at Cosmic Noon**  |
|| L. A. Boogaard, et al. -- incl., <mark>F. Walter</mark>, <mark>S. Bosman</mark> |
|*Appeared on*| *2023-09-01*|
|*Comments*| *19 pages, 10 figures, 1 table, submitted to ApJ*|
|**Abstract**| We present deep James Webb Space Telescope (JWST)/MIRI F560W observations of a flux-limited, ALMA-selected sample of 28 galaxies at z=0.5-3.6 in the Hubble Ultra Deep Field (HUDF). The data from the MIRI Deep Imaging Survey (MIDIS) reveal the stellar structure of the HUDF galaxies at rest-wavelengths of >1 micron for the first time. We revise the stellar mass estimates using new JWST photometry and find good agreement with pre-JWST analysis; the few discrepancies can be explained by blending issues in the earlier lower-resolution Spitzer data. At z~2.5, the resolved rest-frame near-infrared (1.6 micron) structure of the galaxies is significantly more smooth and centrally concentrated than seen by HST at rest-frame 450 nm (F160W), with effective radii of Re(F560W)=1-5 kpc and S\'ersic indices mostly close to an exponential (disk-like) profile (n~1), up to n~5 (excluding AGN). We find an average size ratio of Re(F560W)/Re(F160W)~0.7 that decreases with stellar mass. The stellar structure of the ALMA-selected galaxies is indistinguishable from a HUDF reference sample of galaxies with comparable MIRI flux density. We supplement our analysis with custom-made, position-dependent, empirical PSF models for the F560W observations. The results imply that an older and smoother stellar structure is in place in massive gas-rich, star-forming galaxies at Cosmic Noon, despite a more clumpy rest-frame optical appearance, placing additional constraints on galaxy formation simulations. As a next step, matched-resolution, resolved ALMA observations will be crucial to further link the mass- and light-weighted galaxy structures to the dusty interstellar medium. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2308.16239-b31b1b.svg)](https://arxiv.org/abs/arXiv:2308.16239) | **Bars and boxy/peanut bulges in thin and thick discs III. Boxy/peanut  bulge formation and evolution in presence of thick discs**  |
|| <mark>S. Ghosh</mark>, F. Fragkoudi, P. D. Matteo, K. Saha |
|*Appeared on*| *2023-09-01*|
|*Comments*| *18 pages, 1 table, 17 figures (including appendix), comments are welcome!*|
|**Abstract**| Boxy/peanut (b/p) bulges, the vertically extended inner part of a bar, are ubiquitous in barred disc galaxies in the local Universe, including our own Milky Way. A majority of external galaxies and the Milky Way also possess a thick-disc. However, the dynamical effect of thick-discs in the b/p formation and evolution is not fully understood. Here, we investigate the effect of thick-discs in the formation and evolution of b/p by using a suite of $N$-body models of (kinematically cold) thin and (kinematically hot) thick discs. Within the suite of models, we systematically vary the mass fraction of the thick disc, and the thin-to-thick disc scale length ratio. This allows one to examine the b/p formation in discs with different ratios of cold and hot disc components. The b/ps form in almost all our models via a vertical buckling instability, even in the presence of a massive thick disc. The thin disc b/p is much stronger than the thick disc b/p. With increasing thick disc mass fraction, the final b/p structure gets progressively weaker in strength and larger in extent. Furthermore, the time-interval between the bar formation and the onset of buckling instability gets progressively shorter with increasing thick-disc mass fraction. These trends remain true for all three geometric configurations considered here. The breaking and restoration of the vertical symmetry (during and after the b/p formation) show a spatial variation -- the inner bar region restores vertical symmetry rather quickly (after the buckling) while in the outer bar region, the vertical asymmetry persists long after the buckling happens. Our findings also predict that at higher redshifts, when discs are thought to be thicker, b/ps would have more `boxy-shaped' appearance than more `X-shaped' appearance. This remains to be tested from future observations at higher redshifts. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error list index out of range</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2308.16895.md
    + _build/html/tmp_2308.16895/./rgb_cutouts.png
    + _build/html/tmp_2308.16895/./galaxies_galfit.png
    + _build/html/tmp_2308.16895/./psf2.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\sectionautorefname}{\S}$
$\newcommand{\subsectionautorefname}{\S}$
$\newcommand{\subsubsectionautorefname}{\S}$
$\newcommand{\figureautorefname}{Fig.}$</div>



<div id="title">

# MIDIS: JWST/MIRI reveals the Stellar Structure of ALMA-selected Galaxies in the Hubble--UDF at Cosmic Noon

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2308.16895-b31b1b.svg)](https://arxiv.org/abs/2308.16895)<mark>Appeared on: 2023-09-01</mark> -  _19 pages, 10 figures, 1 table, submitted to ApJ_

</div>
<div id="authors">

L. A. Boogaard, et al. -- incl., <mark>F. Walter</mark>, <mark>S. Bosman</mark>

</div>
<div id="abstract">

**Abstract:** We present deep James Webb Space Telescope (JWST)/MIRI F560W  observations of a flux-limited, ALMA-selected sample of 28 galaxies  at $z=0.5$ -- $3.6$ in the Hubble Ultra Deep Field (HUDF).  The data  from the MIRI Deep Imaging Survey (MIDIS) reveal the stellar  structure of the HUDF galaxies at rest-wavelengths of $\lambda>1$ $\micron$ for the first time.  We revise the stellar  mass estimates using new JWST photometry and find good agreement  with pre-JWST analysis; the few discrepancies can be explained by  blending issues in the earlier lower-resolution Spitzer data.  At $z\sim2.5$ , the resolved rest-frame near-infrared (1.6 $\micron$ )  structure of the galaxies is significantly more smooth and centrally  concentrated than seen by HST at rest-frame 450 nm (F160W), with  effective radii of $\Remiri=1$ --5 kpc and Sérsic indices mostly  close to an exponential (disk-like) profile ( $n\approx1$ ), up to $n\approx5$ (excluding AGN).  We find an average size ratio of $\Remiri/\Rehst\approx0.7$ that decreases with stellar mass. The  stellar structure of the ALMA-selected galaxies is indistinguishable  from a HUDF reference sample of galaxies with comparable MIRI flux  density.  We supplement our analysis with custom-made,  position-dependent, empirical PSF models for the F560W  observations. The results imply that an older and smoother stellar  structure is in place in massive gas-rich, star-forming galaxies at  Cosmic Noon, despite a more clumpy rest-frame optical appearance,  placing additional constraints on galaxy formation simulations.  As  a next step, matched-resolution, resolved ALMA observations will be  crucial to further link the mass- and light-weighted galaxy  structures to the dusty interstellar medium.

</div>

<div id="div_fig1">

<img src="tmp_2308.16895/./rgb_cutouts.png" alt="Fig3" width="100%"/>

**Figure 3. -** MIRI/F560W, NIRCam/F182M, HST/F814W (RGB) cutouts of the
    flux-limited ALMA/ASPECS sample in the MIRI Deep Imaging Survey
    footprint.  The cutouts are ordered by decreasing redshifts and
    are $4$\arcsec$\times4$\arcsec$$, except the last four galaxies at
    $z<1$, which are $8$\arcsec$\times8$\arcsec$$(as indicated by the
    scalebars). See \autoref{tab:sources} for more information on the
    source properties. (*fig:cutouts*)

</div>
<div id="div_fig2">

<img src="tmp_2308.16895/./galaxies_galfit.png" alt="Fig5" width="100%"/>

**Figure 5. -** ALMA galaxies in context of the galaxy population in the Hubble Ultra Deep Field covered by both MIDIS and ASPECS.  The panels show the MIRI/F560W flux density (_left_), effective radius (_center_) and Sérsic index (_right_) as a function of redshift.  The black box in the left panel denotes the HUDF reference sample of galaxies with a flux density in F560W $\geq 1.0$$\mu$Jy, shown in the other panels.
   (*fig:refsample-z-vs-mag*)

</div>
<div id="div_fig3">

<img src="tmp_2308.16895/./psf2.png" alt="Fig8" width="100%"/>

**Figure 8. -** Model of the MIRI/F560 PSF variation over the MIDIS field.  For each of the 96 exposures, the red, green and blue areas in the left panel indicate where the three different stacked PSFs are applicable and the white star marks the location of the star in the image (see \autoref{fig:psf-model}).  The background color map shows the 9 unique areas where the pipeline-processed PSF, which is created by inserting the different PSF model in each exposure at the marked coordinates, is effective. The right panels show a $16$\farcs$5\times16$\farcs$5$ cutout of the pipeline-processed PSFs at the 9 locations.
     (*fig:psf-insert*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2308.16895"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

259  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
